In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=6, suppress=True)

# Решение задачи Дирихле:
## Известно, что $u(x,y) = 2x^3y^3$
## Оно является решением задачи Дирихле:
### $\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = -(\underbrace{-12xy(x^2+y^2)}_{f(x,y)})$
### $u(0,y) \equiv 0$
### $u(1,y) = 2y^3$
### $u(x,0) \equiv 0$
### $u(x,1) = 2x^3$

In [2]:
N = 6
a_x = 0
b_x = 1
a_y = 0
b_y = 1
h = (b_x-a_x)/(N - 1) # ширина сетки вместе с граничными узлами
iN = N - 2 # ширина сетки без граничных узлов

f = lambda x,y: -12*x*y*(x*x+y*y)
u1y = lambda y: 2*y**3
ux1 = lambda x: 2*x**3
u_expected = lambda x,y: 2*x**3*y**3

### Задаём начальное приближение с помощью начальных данных

In [3]:
def init_U(N):
    U = np.zeros(shape = (N, N))
    for i in range(N):
        U[i, N - 1] = ux1(i*h)
    for j in range(N):
        U[N - 1, j] = u1y(j*h)
    return U

def build_init_expected_and_F(N, h, u_expected, ux1, u1y, f):
    U = init_U(N)

    U_expected = U.copy() # задаём границы
    U_expected[1:N-1,1:N-1] = np.array([[u_expected(i*h,j*h) for j in range(1, N-1)] for i in range(1, N-1)])
    F = np.array([[f(i*h, j*h) for j in range(1, N-1)] for i in range(1,N-1)])
    return {'init': U, 'expected': U_expected, 'F': F}

In [4]:
class DESolver:
    def __init__(self, grid_step, N, eps = 1e-5):
        self.h = grid_step
        self.methods = {'iteration': self.iteration_method_iter,
                        'seidel': self.seidel_method_iter,
                        'over-relaxation': self.overrelaxation_method_iter}
        self.methods_params = {'iteration'      : {'min_iters': 2*np.log(1/eps)/(np.pi*h)**2,
                                                'spectral_rad': np.cos(np.pi * h)},
                               'seidel'         : {'min_iters': np.log(1/eps)/(np.pi*h)**2,
                                                'spectral_rad': np.cos(np.pi * h)**2},
                               'over-relaxation': {'min_iters': 2*np.log(1/eps)/(np.pi*h),
                                                'spectral_rad': np.cos(np.pi * h),
                                                'opt_parameter': 2/(1 + np.sin(np.pi * h))}}
        self.N = N
        self.iN = N - 2
        C = np.matrix([[4, -1] + (N-2)*[0]] +
              [ k*[0] + [-1, 4, -1] + (N - 3 - k)*[0] for k in range(0, N - 2)] +
              [(N-2)*[0] + [-1, 4]], dtype=float)
        self.mmatrix = np.block([[C, -np.eye(N = N)] + (N-2)*[np.zeros( shape = (N, N) )]] +
              [ k*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C, -np.eye(N = N)] + (N - 3 - k)*[np.zeros( shape = (N, N) )] for k in range(0, N - 2)] +
              [(N-2)*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C]]) / grid_step**2
        self.lastsol = np.zeros(shape = (N, N))
        self.iter_diffs = np.zeros(shape = 3) # Хранит разницу приближений последних трёх итераций

    def __method_iter(self, U_curr, F, method_name, method_params):
        return self.methods[method_name](U_curr, F, params = method_params)

    def iteration_method_iter(self, U, F, params = None):
        for i in range(1, self.iN+1):
            for j in range(1, self.iN+1):
               self.lastsol[i, j] = 0.25*(U[i-1, j]+U[i+1,j]+U[i, j-1]+U[i, j+1]+F[i-1,j-1]*self.h**2)
        return self.lastsol

    def seidel_method_iter(self, U, F,params = None):
        for k in range(1, self.iN+1):
            for i in range(k, self.iN+1):
                self.lastsol[i, k] = 0.25*(self.lastsol[i-1, k]+U[i+1,k]+self.lastsol[i, k-1]+U[i, k+1]+F[i-1,k-1]*self.h**2)
            for j in range(k+1, self.iN+1):
                self.lastsol[k, j] = 0.25*(self.lastsol[k-1, j]+U[k+1,j]+self.lastsol[k, j-1]+U[k, j+1]+F[k-1,j-1]*self.h**2)
        return self.lastsol

    def overrelaxation_method_iter(self, U, F,params):
            for k in range(1, self.iN+1):
                for i in range(k, self.iN+1):
                    self.lastsol[i, k] = (1-params['opt'])*U[i,k] + 0.25*params['opt']*(self.lastsol[i-1, k]+U[i+1,k]+self.lastsol[i, k-1]+U[i, k+1]+F[i-1,k-1]*self.h**2)
                for j in range(k+1, self.iN+1):
                    self.lastsol[k, j] = (1-params['opt'])*U[k,j] + 0.25*params['opt']*(self.lastsol[k-1, j]+U[k+1,j]+self.lastsol[k, j-1]+U[k, j+1]+F[k-1,j-1]*self.h**2)
            return self.lastsol

    def __calculate_props(self, U, U_expected, F, iter, init_error, init_discrepancy, apost_est_coef):
        iter_diff = np.max(np.abs(U - self.lastsol))
        self.iter_diffs[iter % 3] = iter_diff
        apost_est = apost_est_coef * iter_diff
        spec_rad_est = ' ' if iter < 2 else np.sqrt(self.iter_diffs[iter % 3]/ self.iter_diffs[(iter - 2) % 3] )

        U_reshaped = np.reshape(self.lastsol, newshape=(self.N**2, 1)) # преобразуем матрицу сетки к вектору
        AU = self.mmatrix @ U_reshaped
        AU_reshaped_cut = np.reshape(AU, newshape=(self.N, self.N))[1:self.iN+1, 1:self.iN+1]

        error = np.max(np.abs(self.lastsol - U_expected))
        discrepancy = np.max(np.abs(AU_reshaped_cut - F))
        rel_discrepancy = discrepancy / init_discrepancy
        rel_error = error / init_error

        return [iter + 1, discrepancy, rel_discrepancy,
                             error, rel_error, iter_diff, apost_est, spec_rad_est]

    def solve(self, U_init, U_expected, F, eps = 1e-5, method = 'iteration', method_parameters = None):
        if method not in ['iteration', 'seidel', 'over-relaxation']:
            return None
        else:
            self.iter_diffs = np.zeros(shape = 3)
            if method_parameters is None and method == 'over-relaxation':
                method_parameters = {'opt': self.methods_params['over-relaxation']['opt_parameter']}

        apost_est_coef = self.methods_params[method]['spectral_rad']/ (1 - self.methods_params[method]['spectral_rad'])
        init_error = np.max(np.abs(U_init - U_expected))
        U = U_init.copy()
        self.lastsol = U.copy()

        U_reshaped = np.reshape(U, newshape=(self.N**2, 1)) # преобразуем матрицу сетки к вектору
        AU = self.mmatrix@U_reshaped
        AU_reshaped_cut = np.reshape(AU, newshape=(self.N, self.N))[1:self.iN+1, 1:self.iN+1]

        init_discrepancy = np.max(np.abs(AU_reshaped_cut - F))
        output_list = list()
        iter = 0
        rel_discrepancy = 1
        rel_error = 1

        while rel_discrepancy > eps and iter < self.methods_params[method]['min_iters']:
            U_new = self.__method_iter(U, F, method, method_parameters)

            curr_iter_props = self.__calculate_props(U, U_expected, F, iter, init_error, init_discrepancy, apost_est_coef)
            output_list+=[ curr_iter_props ]

            U = U_new.copy()
            rel_discrepancy = curr_iter_props[2]
            rel_error = curr_iter_props[4]
            iter+=1

        output_list += [[' ', 'min_iters:', self.methods_params[method]['min_iters'], ' ', ' method:', method, ' ', ' ']]
        return {'solution': self.lastsol ,'output': pd.DataFrame(data = output_list,
                                                      columns = ['k', 'Curr. discr.', 'Rel. discr.', 'Curr. error', 'Rel. error', '$ \lVert U_k - U_{k-1} \rVert $', 'Apost. est.', '$\hat{\rho}_k$']).set_index(keys = 'k')}


In [5]:
solver = DESolver(h, N)
initials = build_init_expected_and_F(N, h, u_expected,ux1, u1y, f)
U_init = initials['init']
U_expected = initials['expected']
F = initials['F']

### Решение методом простой итерации

In [6]:
iter_summary = solver.solve(U_init, method = 'iteration', U_expected = U_expected, F = F)
solU_i = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,9.5968,0.231977,0.170784,0.325745,0.413696,1.752444,
2,4.7984,0.115989,0.106704,0.203522,0.095968,0.406527,
3,2.7964,0.067596,0.063936,0.121948,0.047984,0.203263,0.340571
4,2.0665,0.049952,0.056577,0.107912,0.027964,0.118457,0.539804
5,1.15175,0.027840,0.035912,0.068497,0.020665,0.087538,0.65625
6,1.087919,0.026298,0.032529,0.062045,0.011518,0.048789,0.64177
7,0.681805,0.016481,0.022969,0.043809,0.010879,0.046085,0.725572
8,0.628675,0.015197,0.01988,0.037919,0.006818,0.028882,0.769398
9,0.447178,0.010809,0.015044,0.028694,0.006287,0.026631,0.760177


In [7]:
print(U_expected)
print(solU_i)

[[0.       0.       0.       0.       0.       0.      ]
 [0.       0.000128 0.001024 0.003456 0.008192 0.016   ]
 [0.       0.001024 0.008192 0.027648 0.065536 0.128   ]
 [0.       0.003456 0.027648 0.093312 0.221184 0.432   ]
 [0.       0.008192 0.065536 0.221184 0.524288 1.024   ]
 [0.       0.016    0.128    0.432    1.024    2.      ]]
[[0.       0.       0.       0.       0.       0.      ]
 [0.       0.000123 0.001017 0.003447 0.008187 0.016   ]
 [0.       0.001017 0.008178 0.027636 0.065527 0.128   ]
 [0.       0.003447 0.027636 0.093298 0.221177 0.432   ]
 [0.       0.008187 0.065527 0.221177 0.524283 1.024   ]
 [0.       0.016    0.128    0.432    1.024    2.      ]]


### Решение методом Зейделя

In [8]:
iter_summary = solver.solve(U_init, method = 'seidel', U_expected = U_expected, F = F)
solU_s = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,10.827941,0.261737,0.182341,0.347787,0.433118,0.82051,
2,5.613984,0.135703,0.122326,0.233319,0.11228,0.212706,
3,2.097935,0.050712,0.066318,0.126491,0.056008,0.106104,0.359604
4,1.072662,0.025929,0.04259,0.081234,0.027499,0.052096,0.494892
5,0.729314,0.017629,0.030495,0.058164,0.014845,0.028122,0.514827
6,0.483226,0.011681,0.021372,0.040765,0.009665,0.018309,0.592828
7,0.316756,0.007657,0.014432,0.027526,0.006941,0.013148,0.683769
8,0.20736,0.005012,0.009585,0.018281,0.004847,0.009183,0.708201
9,0.135723,0.003281,0.006317,0.012048,0.003268,0.006191,0.686186


In [9]:
print(U_expected)
print(solU_s)

[[0.       0.       0.       0.       0.       0.      ]
 [0.       0.000128 0.001024 0.003456 0.008192 0.016   ]
 [0.       0.001024 0.008192 0.027648 0.065536 0.128   ]
 [0.       0.003456 0.027648 0.093312 0.221184 0.432   ]
 [0.       0.008192 0.065536 0.221184 0.524288 1.024   ]
 [0.       0.016    0.128    0.432    1.024    2.      ]]
[[0.       0.       0.       0.       0.       0.      ]
 [0.       0.000118 0.001011 0.003446 0.008187 0.016   ]
 [0.       0.001011 0.008175 0.027634 0.065529 0.128   ]
 [0.       0.003446 0.027634 0.093301 0.221178 0.432   ]
 [0.       0.008187 0.065529 0.221178 0.524285 1.024   ]
 [0.       0.016    0.128    0.432    1.024    2.      ]]


### Решение методом верхней релаксации

In [10]:
ovopt = 2/(1 + np.sin(np.pi * h))
iter_summary = solver.solve(U_init, method = 'over-relaxation', U_expected = U_expected, F = F)
solU_or = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,12.818102,0.309844,0.178156,0.339806,0.548197,2.322201,
2,8.068365,0.195031,0.121005,0.230799,0.172424,0.730399,
3,3.882178,0.093841,0.065343,0.124632,0.107656,0.456037,0.44315
4,2.467779,0.059652,0.043363,0.082709,0.051782,0.219351,0.548011
5,1.318445,0.031870,0.020158,0.038447,0.033384,0.141419,0.55687
6,0.8154,0.019710,0.009445,0.018014,0.017576,0.074453,0.582602
7,0.122073,0.002951,0.000877,0.001673,0.010271,0.043508,0.554667
8,0.046935,0.001135,0.000712,0.001357,0.001538,0.006514,0.29578
9,0.006236,0.000151,0.00012,0.000229,0.000591,0.002504,0.239917


In [11]:
print(U_expected)
print(solU_or)

[[0.       0.       0.       0.       0.       0.      ]
 [0.       0.000128 0.001024 0.003456 0.008192 0.016   ]
 [0.       0.001024 0.008192 0.027648 0.065536 0.128   ]
 [0.       0.003456 0.027648 0.093312 0.221184 0.432   ]
 [0.       0.008192 0.065536 0.221184 0.524288 1.024   ]
 [0.       0.016    0.128    0.432    1.024    2.      ]]
[[0.       0.       0.       0.       0.       0.      ]
 [0.       0.000125 0.001021 0.003454 0.008192 0.016   ]
 [0.       0.001021 0.008189 0.027647 0.065536 0.128   ]
 [0.       0.003454 0.027647 0.093311 0.221184 0.432   ]
 [0.       0.008192 0.065536 0.221184 0.524288 1.024   ]
 [0.       0.016    0.128    0.432    1.024    2.      ]]


### Убедимся, что параметр по умолчанию является оптимальным

In [12]:
solver.solve(U_init, method = 'seidel', U_expected = U_expected, F = F, method_parameters={'opt': ovopt-0.1})['output']

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,10.827941,0.261737,0.182341,0.347787,0.433118,0.82051,
2,5.613984,0.135703,0.122326,0.233319,0.11228,0.212706,
3,2.097935,0.050712,0.066318,0.126491,0.056008,0.106104,0.359604
4,1.072662,0.025929,0.04259,0.081234,0.027499,0.052096,0.494892
5,0.729314,0.017629,0.030495,0.058164,0.014845,0.028122,0.514827
6,0.483226,0.011681,0.021372,0.040765,0.009665,0.018309,0.592828
7,0.316756,0.007657,0.014432,0.027526,0.006941,0.013148,0.683769
8,0.20736,0.005012,0.009585,0.018281,0.004847,0.009183,0.708201
9,0.135723,0.003281,0.006317,0.012048,0.003268,0.006191,0.686186


In [13]:
solver.solve(U_init, method = 'seidel', U_expected = U_expected, F = F, method_parameters={'opt': ovopt+0.1})['output']

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,10.827941,0.261737,0.182341,0.347787,0.433118,0.82051,
2,5.613984,0.135703,0.122326,0.233319,0.11228,0.212706,
3,2.097935,0.050712,0.066318,0.126491,0.056008,0.106104,0.359604
4,1.072662,0.025929,0.04259,0.081234,0.027499,0.052096,0.494892
5,0.729314,0.017629,0.030495,0.058164,0.014845,0.028122,0.514827
6,0.483226,0.011681,0.021372,0.040765,0.009665,0.018309,0.592828
7,0.316756,0.007657,0.014432,0.027526,0.006941,0.013148,0.683769
8,0.20736,0.005012,0.009585,0.018281,0.004847,0.009183,0.708201
9,0.135723,0.003281,0.006317,0.012048,0.003268,0.006191,0.686186


### видим, что алгоритму приходится выполнять больше итераций, чтобы достичь нужной точности